In [1]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from pandasql import sqldf
import numpy as np

In [2]:
url = "https://raw.githubusercontent.com/PhiPhuongUyen/Team-5/main/marketing_campaign.csv"
df= pd.read_csv(url, sep="\t")
pysqldf = lambda q: sqldf(q, globals())
df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [3]:
df1=pysqldf("SELECT ID, Year_Birth, Income, Dt_Customer, MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds\
             FROM df")

df1

,ID,Year_Birth,Income,Dt_Customer,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds
0,5524,1957,58138.0,04-09-2012,635,88,546,172,88,88
1,2174,1954,46344.0,08-03-2014,11,1,6,2,1,6
2,4141,1965,71613.0,21-08-2013,426,49,127,111,21,42
3,6182,1984,26646.0,10-02-2014,11,4,20,10,3,5
4,5324,1981,58293.0,19-01-2014,173,43,118,46,27,15
...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,61223.0,13-06-2013,709,43,182,42,118,247
2236,4001,1946,64014.0,10-06-2014,406,0,30,0,0,8
2237,7270,1981,56981.0,25-01-2014,908,48,217,32,12,24
2238,8235,1956,69245.0,24-01-2014,428,30,214,80,30,61


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                2240 non-null   int64  
 1   Year_Birth        2240 non-null   int64  
 2   Income            2216 non-null   float64
 3   Dt_Customer       2240 non-null   object 
 4   MntWines          2240 non-null   int64  
 5   MntFruits         2240 non-null   int64  
 6   MntMeatProducts   2240 non-null   int64  
 7   MntFishProducts   2240 non-null   int64  
 8   MntSweetProducts  2240 non-null   int64  
 9   MntGoldProds      2240 non-null   int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 175.1+ KB


From the above output, we can conclude and note that:

There are missing values in income
Dt_Customer that indicates the date a customer joined the database is not parsed as DateTime
There are some categorical features in our data frame; as there are some features in dtype: object). So we will need to encode them into numeric forms later.
First of all, for the missing values, I am simply going to drop the rows that have missing income values.

In [5]:
df1 = df1.dropna()
len(df1)

2216

In the next bit,we will be performing the following steps to engineer some new features:

- Extract the "Age" of a customer by the "Year_Birth" indicating the birth year of the respective person.
- Create another feature "Spent" indicating the total amount spent by the customer in various categories over the span of two years.
- Dropping some of the redundant features

In [6]:
df1["Age"] = 2023-df1["Year_Birth"]
df1["Spent"] = df1["MntWines"]+ df1["MntFruits"]+ df1["MntMeatProducts"]+ df1["MntFishProducts"]+ df1["MntSweetProducts"]+ df1["MntGoldProds"]
df1=df1.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})
to_drop = ["ID"]
df1 = df1.drop(to_drop, axis=1)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15864\369905042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Age"] = 2023-df1["Year_Birth"]
C:\Users\DELL\AppData\Local\Temp\ipykernel_15864\369905042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Spent"] = df1["MntWines"]+ df1["MntFruits"]+ df1["MntMeatProducts"]+ df1["MntFishProducts"]+ df1["MntSweetProducts"]+ df1["MntGoldProds"]


In [7]:
df.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


We filter the year from Dt_Customer to get the data for a line chart

In [8]:
df1['Dt_Customer'] = pd.to_datetime(df1['Dt_Customer'], format='%d-%m-%Y')
df1['Year'] = df1['Dt_Customer'].dt.year
df1['Year'] = df1['Year'].astype(str)
query5 = "SELECT * FROM df1"
dfl = pysqldf(query5)

df1

,Year_Birth,Income,Dt_Customer,Wines,Fruits,Meat,Fish,Sweets,Gold,Age,Spent,Year
0,1957,58138.0,2012-09-04,635,88,546,172,88,88,66,1617,2012
1,1954,46344.0,2014-03-08,11,1,6,2,1,6,69,27,2014
2,1965,71613.0,2013-08-21,426,49,127,111,21,42,58,776,2013
3,1984,26646.0,2014-02-10,11,4,20,10,3,5,39,53,2014
4,1981,58293.0,2014-01-19,173,43,118,46,27,15,42,422,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1967,61223.0,2013-06-13,709,43,182,42,118,247,56,1341,2013
2236,1946,64014.0,2014-06-10,406,0,30,0,0,8,77,444,2014
2237,1981,56981.0,2014-01-25,908,48,217,32,12,24,42,1241,2014
2238,1956,69245.0,2014-01-24,428,30,214,80,30,61,67,843,2014


In [11]:
query2 = '''
SELECT
    Age,
    COUNT(*) AS count,
    SUM(Wines) AS tl_wines,
    SUM(Fruits) AS tl_frts,
    SUM(Meat) AS tl_meat,
    SUM(Fish) AS tl_fish,
    SUM(Sweets) AS tl_sweet,
    SUM(Gold) AS tl_gold
FROM 
    (SELECT 
        CASE
            WHEN Age BETWEEN 18 AND 30 THEN '18-30'
            WHEN Age BETWEEN 30 AND 49 THEN '30-49'
            WHEN Age BETWEEN 50 AND 64 THEN '50-64'
            WHEN Age BETWEEN 65 AND 75 THEN '65-75'
            WHEN Age BETWEEN 75 AND 89 THEN '75-89'
            WHEN Age BETWEEN 90 AND 100 THEN '90-100'
            ELSE 'Other'
        END AS Age,
            Wines, 
            Fruits, 
            Meat, 
            Fish, 
            Sweets, 
            Gold
    FROM 
        df1) AS subquery
GROUP BY 
    Age;
'''

dfcl1 = sqldf(query2, locals())

dfcl1

,Age,count,tl_wines,tl_frts,tl_meat,tl_fish,tl_sweet,tl_gold
0,18-30,15,5357,649,5127,1406,691,1042
1,30-49,826,202450,21170,129077,29659,21945,32879
2,50-64,863,272449,22574,135849,29863,21941,37472
3,65-75,454,167373,11560,83197,18908,13064,23312
4,75-89,55,27676,2302,16238,3451,2187,2471
5,Other,3,778,150,575,118,68,251


In [12]:
df_liner=pysqldf("SELECT Age, Income, Wines, Fruits, Meat, Fish, Sweets, Gold, Spent\
             FROM df1\
             GROUP BY Age\
             ORDER BY Age ASC")
df_liner

,Age,Income,Wines,Fruits,Meat,Fish,Sweets,Gold,Spent
0,27,7500.0,3,18,14,15,22,50,122
1,28,34824.0,4,2,11,2,0,4,23
2,29,80134.0,966,26,282,52,26,26,1378
3,30,74293.0,375,152,335,93,91,81,1127
4,31,92859.0,962,61,921,52,61,20,2077
5,32,68126.0,1332,17,311,23,51,86,1820
6,33,73687.0,559,153,503,218,13,181,1627
7,34,38360.0,36,2,42,20,21,10,131
8,35,27938.0,13,0,8,2,1,5,29
9,36,17323.0,3,14,17,6,1,5,46


In [25]:
from sklearn.linear_model import LinearRegression
est = LinearRegression(fit_intercept=True)
x=df_liner[['Wines','Fruits','Meat', 'Fish', 'Sweets', 'Gold']]
y=df_liner[['Income']]
est.fit(x,y)
print(est.coef_)
print(est.intercept_)
print("R^2 is ",est.score(pd.DataFrame(X_test),pd.DataFrame(y_test)))

[[ 46.93021094 176.38438793 -13.06163772  -0.28823358 133.77590578
  -65.05377997]]
[32295.44844568]
R^2 is  0.7618912813076114


The R^2 is an evaluation metric that measures the goodness of fit of the model to the data. It represents the proportion of the variance in the target variable that can be explained by the model. R^2 values range from 0 to 1, and a value closer to 1 indicates a better fit of the model to the data. And our R^2 is 0.7618912813076114 so we can see that the model was fit to the data.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
est = LinearRegression(fit_intercept=True)
est.fit(pd.DataFrame(X_train),pd.DataFrame(y_train))

LinearRegression()

In [15]:
import statsmodels.api as sm
X_train2 = sm.add_constant(X_train) # This adds an intercept
model = sm.OLS(y_train,X_train2)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Income   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     18.36
Date:                Sun, 08 Oct 2023   Prob (F-statistic):           4.18e-10
Time:                        17:38:12   Log-Likelihood:                -507.96
No. Observations:                  47   AIC:                             1030.
Df Residuals:                      40   BIC:                             1043.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.227e+04   2864.627     11.267      0.000    2.65e+04    3.81e+04
Wines         43.7058      6.814      6.414      0.000      29.935      57.477
Fruits       172.3178     71.442      2.412      0.021      27.928     316.708
Meat         -13.7021      6.939     -1.975      0.055     -27.727       0.323
Fish           1.2796     44.220      0.029      0.977     -88.093      90.652
Sweets       142.1628     57.648      2.466      0.018      25.652     258.674
Gold         -50.8173     48.952     -1.038      0.305    -149.753      48.118
==============================================================================
Omnibus:                        2.627   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.269   Jarque-Bera (JB):                1.705
Skew:                          -0.428   Prob(JB):                        0.426
Kurtosis:                       3.371   Cond. No.                         884.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Coefficients: The coefficients of the model indicate the relationship between the independent variables and the target variable. In linear regression, these coefficients represent the change in the target variable for a one-unit change in the corresponding independent variable. A higher magnitude coefficient suggests a stronger impact on the target variable.

Intercept: The intercept represents the estimated target variable value when all independent variables are zero. It provides a baseline reference point for the model's predictions.

The p-value is 4.18e-10, which is a very small value (smaller than 0.05 or 0.01), indicating that the F-statistic is highly statistically significant. This suggests that the overall regression model has a significant relationship with the dependent variable, and at least one independent variable in the model is contributing significantly to explaining the variation in the dependent variable. The regression model is statistically significant and provides a good fit for the data.

R-squared (R2) Score: R2 score measures the proportion of the variance in the target variable that can be explained by the independent variables. It ranges from 0 to 1, with higher values indicating a better fit. R2 score of 1 means the model explains all the variance in the target variable, while a score of 0 means the model fails to explain any variance.

In [18]:
dfclean1=pysqldf("SELECT COUNT(Age), Age\
             FROM df1\
             GROUP BY Age\
             ORDER BY Age ASC")
dfclean1.head()

,COUNT(Age),Age
0,2,27
1,5,28
2,3,29
3,5,30
4,13,31
